In [71]:
import re

def extract_resume_sections(file_path):
    """
    Extract only LaTeX resume sections, skipping preamble and personal info.
    Returns a list of sections as strings.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Ensure document starts correctly
    if "\\begin{document}" not in content:
        raise ValueError("No \\begin{document} found in LaTeX file.")

    # print(content.split("\\begin{document}", 1)[0])
    # Extract everything after \begin{document}
    body_start = content.split("\\begin{document}", 1)[1]

    # Extract everything before \end{document}, if present
    body, sep, after = body_start.partition("\\end{document}")

    # Skip personal info: start from first \section or \section*
    first_section_match = re.search(r"\\section\*?|\\section", body)
    if not first_section_match:
        raise ValueError("No \\section found in LaTeX resume body.")

    sections_text = body[first_section_match.start():]

    # Split by section headers
    sections = re.split(r"(?=\\section\*?|\\section)", sections_text)
    sections = [sec.strip() for sec in sections if sec.strip()]

    return sections




if __name__ == "__main__":
    resume_path = '../Application_All/Resume/Resume_Ankit.tex'
    sections = extract_resume_sections(resume_path)
    
    # print("\nSections:")
    # for section in sections:
    #     print(section)
    

In [72]:
from Resume import ResumeModifier

In [73]:
sections = [sec for sec in sections]


In [74]:
import re

def extract_intro_and_sections(file_path):
    """
    Extract the personal info / intro and the sections separately from a LaTeX resume.
    Returns:
        intro: str (personal info at top)
        sections: list of strings (each LaTeX section)
    """
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Everything between \begin{document} and \end{document}
    body_start = content.split("\\begin{document}", 1)[1]
    body, _, _ = body_start.partition("\\end{document}")

    # Split into intro (everything before first section) and sections
    match = re.search(r"\\section\*?|\\section", body)
    if not match:
        raise ValueError("No sections found in resume.")

    intro = body[:match.start()].strip()  # personal info
    sections_text = body[match.start():]

    # Split sections by \section or \section*
    sections = re.split(r"(?=\\section\*?|\\section)", sections_text)
    sections = [sec.strip() for sec in sections if sec.strip()]

    return intro, sections


In [75]:
def assemble_resume(intro, sections, original_file="resume.tex", output_file="tailored_resume.tex"):
    """
    Reassemble LaTeX resume keeping preamble, intro (personal info), and modified sections.
    """
    with open(original_file, "r", encoding="utf-8") as f:
        content = f.read()

    preamble, _, _ = content.partition("\\begin{document}")
    _, sep, end = content.rpartition("\\end{document}")
    end_document = sep + end if sep else "\\end{document}"

    # Join sections safely
    new_content = preamble + "\\begin{document}\n\n" + intro + "\n\n" + "\n\n".join(sections) + "\n\n" + end_document

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(new_content)

    print(f"Tailored resume saved to {output_file}")


resume_path = '../Application_All/Resume/Resume_Ankit.tex'


# 1️⃣ Extract
intro, sections = extract_intro_and_sections(resume_path)

# # 2️⃣ Tailor sections using your model
# tailored_sections = tailor_sections_with_model(sections, job_desc)

# 3️⃣ Assemble final LaTeX resume
assemble_resume(intro, sections, original_file=resume_path, output_file="tailored_resume.tex")


Tailored resume saved to tailored_resume.tex


In [1]:
from Resume import ResumeModifier

resume = (ResumeModifier(
            resume_path='../Application_All/Resume/Resume_Ankit.tex',
            job_description='./job_description.txt',  # Add job description if available
            outdir='./test',  # Specify output directory if needed
            company_name='test'  # Specify company name if needed,
            )
        .modify_resume(debug=False)
        )

Extracting sections from resume...
Tailoring sections with the model...
Tailoring section 1
Tailoring section 2
Tailoring section 3
Tailoring section 4
Tailoring section 5
Tailoring section 6
Tailoring section 7
Assembling the final resume...


Latexmk: This is Latexmk, John Collins, 26 Dec. 2019, version: 4.67.
Latexmk: Changing directory to 'test/'
Rule 'pdflatex': The following rules & subrules became out-of-date:
      'pdflatex'
------------
Run number 1 of rule 'pdflatex'
------------
------------
Running 'pdflatex  -recorder  "test_resume.tex"'
------------


Latexmk: applying rule 'pdflatex'...
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./test_resume.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size11.clo))
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/share/texlive/texmf-dist/tex/generic/iftex/iftex.sty)))
(/usr/share/texlive/texmf-dist/tex/latex/base/latexsym.sty)
(/usr/share/texlive/texmf-dist/tex/latex/preprint/fullpage.sty)
(/usr/share/texlive/texmf-dist/tex/latex/titlesec/titlesec.sty)
(/usr/share/texlive/texmf-dist/tex/latex/marvosym/marvosym.sty)
(/usr/share/texlive/texmf-dist/tex/la

KeyboardInterrupt: 

In [6]:
from concurrent.futures import ThreadPoolExecutor
from langchain_core.runnables import RunnableConfig
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnableParallel

# A. Limit Python‐level parallelism
# executor = ThreadPoolExecutor(max_workers=2)
# config   = RunnableConfig(executor=executor)

# B. Limit Ollama’s internal threads
llm = ChatOllama(
    model="gemma:2b",
    validate_model_on_init=True,
    num_thread=4
)

job_description = """
Looking for a data scientist with strong Python, SQL, and machine learning skills,
able to handle large datasets and time series analysis.
"""

# Dummy sections
sections_dict = {
    "0": r"""\resumeItem{\textbf{Option Pricing Models:} Implemented Binomial Tree, Black-Scholes, and Monte Carlo simulations for derivatives pricing. \href{https://github.com/ansingh16/derivative_pricing.git}{\underline{GitHub}}}
\resumeItem{\textbf{Backtesting Framework:} Developed an OOP-based simulation for stock trading strategies. Incorporated volatility-adjusted signals using GARCH. \href{https://github.com/ansingh16/Backtesting}{\underline{GitHub}}}
\resumeItem{\textbf{Portfolio Optimization:} Applied Modern Portfolio Theory, Monte Carlo simulations, and convex optimization to model efficient frontiers. \href{https://github.com/ansingh16/portfolio-optimization}{\underline{GitHub}}}
\resumeItem{\textbf{Risk Management Toolkit:} Created technical indicator functions, Sharpe/Alpha/Beta calculators, and VaR estimators with signal denoising filters. \href{https://github.com/ansingh16/risk-management}{\underline{GitHub}}}
\resumeItem{\textbf{Time Series Forecasting:} Modeled equity prices using ARIMA and GARCH, with rolling window predictions and diagnostic dashboards. \href{https://github.com/ansingh16/Time_Series_Analysis}{\underline{GitHub}}}
\resumeItem{\textbf{Mutual Fund Analyzer:} Built a Streamlit app integrating public APIs to assess diversification, Sharpe ratio, and sector exposure. \href{https://mf-analysis.streamlit.app/}{\underline{App}} \href{https://github.com/ansingh16/MF_analysis}{\underline{GitHub}}}
\resumeItemListEnd
""",
    "1": "Skills: Python, SQL, Machine Learning, Time Series Analysis.",
    "2": "Education: PhD in Physics with focus on data analysis."
}

prompt = ChatPromptTemplate.from_template(
    """
    You are an expert resume editor.
    Job description:
    {job_description}

    Edit the following LaTeX-style section  
    to align with the job description.  
    Only modify content; do NOT change LaTeX commands.

    Section:
    {section}

    Edited section:
    """
).partial(job_description=job_description)

chain = prompt | llm

keys         = list(sections_dict.keys())
batch_inputs = [{"section": sections_dict[k]} for k in keys]

# Run with both caps in place
batch_responses = chain.batch(batch_inputs,config={'max_concurrency': 4})

results = {k: ai_msg.content for k, ai_msg in zip(keys, batch_responses)}
for k, text in results.items():
    print(f"Edited section {k}:\n{text}\n")

Edited section 0:
    \resumeItem{\textbf{Option Pricing Models:} Implemented Binomial Tree, Black-Scholes, and Monte Carlo simulations for derivative pricing. \href{https://github.com/ansingh16/derivative_pricing.git}{\underline{GitHub}}}
\resumeItem{\textbf{Backtesting Framework:} Developed an OOP-based simulation for stock trading strategies. Incorporated volatility-adjusted signals using GARCH. \href{https://github.com/ansingh16/Backtesting}{\underline{GitHub}}}
\resumeItem{\textbf{Portfolio Optimization:} Applied Modern Portfolio Theory, Monte Carlo simulations, and convex optimization to model efficient frontiers. \href{https://github.com/ansingh16/portfolio-optimization}{\underline{GitHub}}}
\resumeItem{\textbf{Risk Management Toolkit:} Created technical indicator functions, Sharpe/Alpha/Beta calculators, and VaR estimators with signal denoising filters. \href{https://github.com/ansingh16/risk-management}{\underline{GitHub}}}
\resumeItem{\textbf{Time Series Forecasting:} Modeled